In [29]:
import util

meta_init_prompt = """Give 50 distinct textual descriptions of pairs of visual discriminative features to identify whether a dermoscopic image shows melanoma or not. 
Only provide the output as Python code in the following format: prompts = list[tuple[negative: str, positive: str]]. Let's think step-by-step"""
client = util.LLMClient(provider='gemini')
response = client.get_llm_response(meta_init_prompt)
print(response)


```python
prompts = [
    ("Uniform pigmentation throughout the lesion.", "Asymmetrical pigmentation with varying shades of brown and black."),
    ("Well-defined, smooth borders.", "Irregular, notched, or blurred borders."),
    ("Consistent skin markings around the lesion.", "Loss of surrounding skin markings, such as skin lines."),
    ("Small diameter, less than 6mm.", "Large diameter, greater than 6mm."),
    ("Round or oval shape.", "Elliptical, irregular, or amorphous shape."),
    ("No visible vascular patterns.", "Presence of atypical vascular structures like dotted vessels or arborizing vessels."),
    ("Uniform color.", "Multiple colors within the lesion, including brown, black, red, white, and blue."),
    ("No elevation from the skin surface.", "Raised or palpable elevation from the skin surface."),
    ("No ulceration or bleeding.", "Ulceration, bleeding, or crusting on the lesion surface."),
    ("No satellite lesions.", "Presence of small, secondary lesions around the p

# multi-class prompt

In [36]:
import util

multiclass_prompt = """Give 20 distinct textual description sets to identify the pigment network of a dermoscopic image. Pigment Networks are labelled as absent, typical or atypical. 
Each description set must contain three distinct, contrasting features: one for an absent, one for a typical, and one for an atypical pigment network. These features should be direct, discriminating characteristics, not independent descriptions. You should NOT include any observations related to Blue Whitish Veil, Vascular Structures, Pigmentation, Streaks, Dots and Globules, Regression Structures. Only focus on the Pigment Network.
Only provide the output as Python code in the following format: prompts = list[tuple[str, str, str]]. Let's think step-by-step"""
client = util.LLMClient(provider='gemini')
response = client.get_llm_response(multiclass_prompt)
print(response)


```python
prompts = [
    ("No discernible network present; skin appears uniformly colored.", "Regular, reticular network with consistent mesh size and dark borders.", "Irregular network with varying mesh sizes and ill-defined borders."),
    ("Lack of any organized pigmented structures.", "Fine, delicate network with evenly spaced holes.", "Coarse, thick network with large, irregular holes."),
    ("Homogeneous background color without any network pattern.", "Network lines are straight and parallel, forming a regular grid.", "Network lines are curved and branching, creating an amorphous pattern."),
    ("Diffuse pigmentation obscures any potential network.", "Network is symmetrical and evenly distributed across the lesion.", "Network is asymmetrical, with denser areas in certain regions."),
    ("No visible lines or areas of increased pigmentation forming a network.", "Network edges are sharp and well-defined.", "Network edges are blurred and indistinct."),
    ("Absence of any repeat

In [33]:
import util

# Prepare the chat prompt
base_meta_prompt_template = """The task is to generate distinct textual description sets to identify the pigment network of a dermoscopic image. Pigment Networks are labelled as absent, typical or atypical. 
Each description set must contain three distinct, contrasting features: one for an absent, one for a typical, and one for an atypical pigment network. These features should be direct, discriminating characteristics, not independent descriptions. You should only describe the pigment network aspect of the dermoscopic image.
Here are the best performing pairs in ascending order. High scores indicate higher quality visual discriminative features.
{content}
{iteration_specific_instruction}
Only provide the output as Python code in the following format: prompts = list[tuple[str, str, str]]. Think step-by-step.
"""
content = """Current Top 10 prompts:
1. "No discernible network structures are present; skin appears uniformly colored.", "Regular, reticular network with even spacing and consistent mesh size.", "Irregular network with varying mesh sizes, blurred borders, and absent peripheral network." Score: 60
2. "Pigment is distributed diffusely without any organized pattern.", "Fine, delicate network with short streaks and a lace-like appearance.", "Coarse, thick network with prominent streaks and irregular spacing, exhibiting radial streaming." Score: 61
3. "Background color dominates, obscuring any potential network.", "Well-defined network with consistent pigmentation throughout.", "Network is fragmented and distorted, with areas of hypopigmentation and hyperpigmentation." Score: 65
4. "No lines or dots are visible suggesting a network.", "Regular network with a honeycomb-like pattern.", "Atypical network with branching streaks and a starburst pattern." Score: 69
5. "Uniformly tan or pink skin with no network features.", "Network appears as a regular grid with consistent spacing.", "Network is asymmetrical and appears stretched or compressed in certain areas." Score: 72
6. "Absence of any pigmented structures resembling a network.", "Network is symmetrical and evenly distributed.", "Network shows abrupt changes in direction and thickness." Score: 75
7. "Skin appears homogenous with no visible network.", "Network is composed of short, fine lines forming a regular pattern.", "Network is characterized by large, irregular blotches and indistinct borders." Scpore: 77
8. "No evidence of a pigment network can be identified.", "Network is clearly visible and follows a consistent pattern.", "Network is disrupted by areas of regression or scarring." Score: 85
9. "Diffuse pigmentation without any organized network.", "Network is regular and well-defined with consistent pigmentation.", "Network is atypical with a combination of reticular and globular patterns." Score: 90
10. "No network is visible; skin appears uniformly pigmented.", "Network is regular and well-defined with consistent spacing.", "Network is atypical with a combination of reticular and globular patterns and regression structures." Score: 95
"""
iteration_specific_instruction = """Write {generate_n} new prompt pairs that are different to from the old ones and has a score as high as possible. Formulate a strategy. Let's think step-by-step."""

prompt = base_meta_prompt_template.format(
    content=content,
    iteration_specific_instruction=iteration_specific_instruction.format(generate_n=10)
)

client = util.LLMClient(provider='gemini')
response = client.get_llm_response(prompt)
print(response)


Okay, let's analyze the top-performing prompts and formulate a strategy to generate new, high-scoring ones.

**Strategy:**

1.  **Focus on Contrast:** The best prompts have *very* distinct features for absent, typical, and atypical networks. We need to maintain this.
2.  **Specificity:** Avoid vague terms like "irregular" without further qualification. Instead, describe *how* it's irregular (e.g., "varying mesh sizes," "blurred borders").
3.  **Key Dermoscopic Features:** Leverage known dermoscopic features associated with pigment networks:
    *   **Absent:** Diffuse pigmentation, homogenous skin, lack of lines/dots.
    *   **Typical:** Reticular (net-like), regular spacing, consistent pigmentation, fine lines.
    *   **Atypical:**  Globular, branching streaks, radial streaming, asymmetry, abrupt changes, regression structures, blotches, distorted patterns, combination of patterns.
4.  **Avoid Redundancy:**  Ensure the new prompts don't simply rephrase existing ones.
5.  **Regressio